# Before you start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
import pandas as pd
import numpy as np

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents. Please, load the data using Ironhack's database (db: employees, table: employees_advanced).

In [2]:
salaries = pd.read_csv("../data/hypothesis-testing.csv")


Examine the `salaries` dataset using the `head` function below.

In [3]:
salaries.head()


,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [4]:
salaries.isnull().sum(axis = 0)

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [5]:
salaries['Salary or Hourly'].value_counts()



Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [6]:
salaries.groupby('Department')['Name'].agg('count')

Department
ADMIN HEARNG                39
ANIMAL CONTRL               81
AVIATION                  1629
BOARD OF ELECTION          107
BOARD OF ETHICS              8
BUDGET & MGMT               46
BUILDINGS                  269
BUSINESS AFFAIRS           171
CITY CLERK                  84
CITY COUNCIL               411
COMMUNITY DEVELOPMENT      207
COPA                       116
CULTURAL AFFAIRS            65
DISABILITIES                28
DoIT                        99
FAMILY & SUPPORT           615
FINANCE                    560
FIRE                      4641
GENERAL SERVICES           980
HEALTH                     488
HUMAN RELATIONS             16
HUMAN RESOURCES             79
INSPECTOR GEN               87
LAW                        407
LICENSE APPL COMM            1
MAYOR'S OFFICE              85
OEMC                      2102
POLICE                   13414
POLICE BOARD                 2
PROCUREMENT                 92
PUBLIC LIBRARY            1015
STREETS & SAN             21

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [7]:
from scipy import stats
from scipy.stats import ttest_1samp, t

In [8]:
hourly_workers = salaries[(salaries['Salary or Hourly'] == 'Hourly')]
hourly_workers.count()

Name                 8022
Job Titles           8022
Department           8022
Full or Part-Time    8022
Salary or Hourly     8022
Typical Hours        8022
Annual Salary           0
Hourly Rate          8022
dtype: int64

In [9]:
'''
H0 = Hourly Rate is  $30/hr.
H1 = Hourly Rate is different than  $30/hr.
'''

mu_hat = hourly_workers['Hourly Rate'].mean()
sigma_hat = hourly_workers['Hourly Rate'].std()
n = hourly_workers['Hourly Rate'].count()
mu = 30

In [10]:
test_results = ttest_1samp(hourly_workers['Hourly Rate'], 30)
display(test_results.statistic)
display(test_results.pvalue)

20.6198057854942

4.3230240486229894e-92

In [11]:
'''p_value es much smaller that 0.05 so the null hypothesis is rejected'''

'p_value es much smaller that 0.05 so the null hypothesis is rejected'

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [12]:
'''
H0 = Policemen salary is $86000 / year
H1 = Policemen salary is higher than $86000 / year
'''

policemen = salaries[(salaries['Department'] == 'POLICE') | (salaries['Department'] == 'POLICE BOARD')]
policemen['Annual Salary'].count()

13406

In [13]:
test_results = ttest_1samp(policemen['Annual Salary'], 86000)
display(test_results.statistic)
display(test_results.pvalue / 2)

#Because of nulls, ttest_1samp result is nan, so i will operate old school.

nan

nan

In [14]:
mu_hat = np.mean(policemen['Annual Salary'])
sigma_hat = np.std(policemen['Annual Salary'])
n = policemen['Annual Salary'].count()
mu = 86000

t_test = (mu_hat - mu) / (sigma_hat/np.sqrt(n)) 
t_test

3.081974392028248

In [15]:
n = policemen['Annual Salary'].count()

In [16]:
rv = t(df=n-1)
p_value = 1 - rv.cdf(t_test)
p_value

0.0010302480735085995

In [17]:
'''p_value es smaller that 0.05 so the null hypothesis is rejected. The chief is correct'''

'p_value es smaller that 0.05 so the null hypothesis is rejected. The chief is correct'

Using the `crosstab` function, find the department that has the most hourly workers. 

In [18]:
worktype_by_dep = pd.crosstab(salaries['Department'], salaries['Salary or Hourly'])
maximum = worktype_by_dep['Hourly'].sort_values(ascending=False).head(1)
print(f"The department that has most hourly workers is {maximum}")

The department that has most hourly workers is Department
STREETS & SAN    1862
Name: Hourly, dtype: int64


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [19]:
'''
H0 = Streets & San workers salary is $35 / hour
H1 = Streets & San workers salary is lower than $35 / hour
'''

SandS_salary = salaries[(salaries['Department'] == 'STREETS & SAN')]
SandS_salary['Hourly Rate'].dropna(inplace=True)
test_results = ttest_1samp(SandS_salary['Hourly Rate'], 35)
display(test_results.statistic)
display(test_results.pvalue/2)

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/pandas/core/series.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(result)


-9.567447887848152

1.6689265282353859e-21

In [20]:
'''p_value es smaller that 0.05 so the null hypothesis is rejected. They make shit money'''

'p_value es smaller that 0.05 so the null hypothesis is rejected. They make shit money'

# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [21]:
hourly_mean = hourly_workers['Hourly Rate'].mean()
hourly_std = stats.sem(hourly_workers['Hourly Rate'])
n = hourly_workers['Hourly Rate'].count()

t.interval(alpha=0.95, df=n-1, loc=hourly_mean, scale=hourly_std) 



(32.52345834488425, 33.05365708767623)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [22]:
policemen['Annual Salary'].dropna(inplace=True)
police_mean = np.mean(policemen['Annual Salary'])
police_std = stats.sem(policemen['Annual Salary'])
n = policemen['Annual Salary'].count()

t.interval(alpha=0.95, df=n-1, loc=police_mean, scale=police_std) 

(86177.02348322631, 86795.70096851172)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [23]:
import statsmodels
from statsmodels.stats.proportion import proportions_ztest

In [44]:
'''
H0 = Proportion of hourly wokers in Chicago is 25%
H1 = Proportion of hourly wokers in Chicago is diferent than 25%
'''

total_ws = salaries.shape[0]
hourly_ws = hourly_workers.shape[0]
value = 0.25
stat, pval = statsmodels.stats.proportion.proportions_ztest(hourly_ws, total_ws, value)
print(stat, pval)

-3.5099964213703005 0.0004481127249057967


In [47]:
'''We can reject null hypothesis, so the proportion of hourly workers is significat different that 25% with a significance level  of 0.05.'''

'We can reject null hypothesis, so the proportion of hourly workers is significat different that 25% with a significance level  of 0.05.'